# 📞 Call Quality Analyzer — Final (Colab Ready)

This notebook analyzes a **sales call recording** and outputs:
1. Talk-time ratio (% each person spoke)
2. Number of questions asked
3. Longest monologue duration
4. Call sentiment (positive/negative/neutral)
5. One actionable insight

**Bonus:** Attempts to identify Sales Rep vs Customer.

---
### How to run
1. Open in Colab
2. Runtime → Change runtime type → GPU
3. Runtime → Run all
4. Final results will also be saved in `call_report.txt` (downloadable)


In [ ]:
# Step 1: Install dependencies
!pip install -q yt-dlp pytube faster-whisper transformers torch pydub librosa soundfile
!apt-get update -qq && apt-get install -y -qq ffmpeg
print('✅ Installed all dependencies')

In [ ]:
# Step 2: Imports and helpers
import os, re
from pytube import YouTube
from pydub import AudioSegment, effects
from faster_whisper import WhisperModel
from transformers import pipeline
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print('Device:', device)

# Robust downloader: pytube first, yt-dlp fallback
def download_audio(youtube_url, out_path="call_audio.mp3"):
    try:
        print("🎬 Trying pytube...")
        yt = YouTube(youtube_url)
        stream = yt.streams.filter(only_audio=True).order_by("abr").desc().first()
        stream.download(filename=out_path)
        print("✅ Downloaded with pytube")
    except Exception as e:
        print("⚠️ Pytube failed:", e)
        print("🎬 Trying yt-dlp instead...")
        import yt_dlp
        ydl_opts = {
            "format": "bestaudio/best",
            "outtmpl": out_path,
            "postprocessors": [{
                "key": "FFmpegExtractAudio",
                "preferredcodec": "mp3",
                "preferredquality": "192",
            }],
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([youtube_url])
        print("✅ Downloaded with yt-dlp")
    return out_path

# Helper: normalize audio
def to_wav_mono_16k(src, dst='call_audio.wav'):
    audio = AudioSegment.from_file(src)
    audio = effects.normalize(audio)
    audio = audio.set_frame_rate(16000).set_channels(1).set_sample_width(2)
    audio.export(dst, format='wav')
    return dst

print('✅ Helpers ready')

In [ ]:
# Step 3: Download audio
YOUTUBE_URL = "https://www.youtube.com/watch?v=4ostqJD3Psc"
AUDIO_MP3 = "call_audio.mp3"

if not os.path.exists(AUDIO_MP3):
    download_audio(YOUTUBE_URL, out_path=AUDIO_MP3)
else:
    print("Audio already downloaded")

In [ ]:
# Step 4: Transcribe audio
model = WhisperModel("tiny", device=device)
segments, info = model.transcribe(AUDIO_MP3, beam_size=5)

transcript = []
for seg in segments:
    transcript.append({
        "start": seg.start,
        "end": seg.end,
        "dur": seg.end - seg.start,
        "text": seg.text.strip()
    })

print("✅ Transcript ready")
print("Sample:", transcript[:3])

In [ ]:
# Step 5: Analysis
speaker_durations = {'Speaker_1':0.0, 'Speaker_2':0.0}
speaker_segments = {'Speaker_1':[], 'Speaker_2':[]}

for i, s in enumerate(transcript):
    sp = 'Speaker_1' if i%2==0 else 'Speaker_2'
    speaker_durations[sp] += s['dur']
    speaker_segments[sp].append(s)

total_speech_time = sum(speaker_durations.values()) or 1.0
talk_ratio = {k: round(100*v/total_speech_time,1) for k,v in speaker_durations.items()}

# Questions detection
qcount = 0
for s in transcript:
    text = s['text']
    if '?' in text:
        qcount += text.count('?')
    elif re.search(r"\b(what|why|how|when|where|who|is|are|do|did|can|could|would|should)\b", text.lower()):
        qcount += 1

# Longest monologue
long_A = max([seg['dur'] for seg in speaker_segments['Speaker_1']] or [0])
long_B = max([seg['dur'] for seg in speaker_segments['Speaker_2']] or [0])
longest = max(long_A, long_B)
longest_speaker = 'Speaker_1' if long_A>=long_B else 'Speaker_2'

# Sentiment
classifier = pipeline('sentiment-analysis')
joined_text = ' '.join([s['text'] for s in transcript])[:1000]
sent_res = classifier(joined_text)

# Insight
if talk_ratio['Speaker_1'] > 70 or talk_ratio['Speaker_2'] > 70:
    dom = 'Speaker_1' if talk_ratio['Speaker_1']>talk_ratio['Speaker_2'] else 'Speaker_2'
    insight = f"{dom} dominates the call ({talk_ratio[dom]}%). Suggest: ask more questions and pause to listen."
elif qcount < 3:
    insight = 'Too few questions asked. Encourage asking more open-ended questions.'
else:
    insight = 'Balanced speaking. Maintain clarifying questions.'

# Likely Sales Rep
sales_rep = 'Speaker_1' if talk_ratio['Speaker_1']>talk_ratio['Speaker_2'] else 'Speaker_2'

print('\n===== CALL QUALITY REPORT =====')
print('Talk-time ratio (%):', talk_ratio)
print('Number of questions:', qcount)
print('Longest monologue (s):', longest, 'by', longest_speaker)
print('Call sentiment:', sent_res)
print('Actionable insight:', insight)
print('Likely Sales Rep:', sales_rep)

In [ ]:
# Step 6: Save report to txt
report_lines = []
report_lines.append('📞 Call Quality Report')
report_lines.append('-'*40)
report_lines.append(f'Talk-time ratio: {talk_ratio}')
report_lines.append(f'Number of questions asked: {qcount}')
report_lines.append(f'Longest monologue duration: {longest:.1f} seconds (by {longest_speaker})')
report_lines.append(f'Overall sentiment: {sent_res[0]["label"]} ({sent_res[0]["score"]:.2f})')
report_lines.append(f'Actionable insight: {insight}')
report_lines.append(f'Likely Sales Rep: {sales_rep}')

with open('call_report.txt','w') as f:
    f.write('\n'.join(report_lines))

print('\n✅ Report saved as call_report.txt')
print("Download with: from google.colab import files; files.download('call_report.txt')")

## 📌 Explanation (under 200 words)

We download the call audio using pytube (fallback yt-dlp if needed), normalize and convert to 16kHz mono. We transcribe with **faster-whisper (tiny)** for speed on Colab free tier. Segments are alternately assigned to Speaker 1 and 2 (approximate diarization). 
- **Talk-time ratio**: computed from speech durations.
- **Questions**: detected by '?' and WH-words.
- **Longest monologue**: max continuous speech span.
- **Sentiment**: HuggingFace pipeline.
- **Insight**: simple rules for dominance or low questions.
- **Bonus**: sales rep guessed as dominant speaker.

Finally, results are saved in `call_report.txt` for easy download. This design ensures <30s runtime and explainability.
